[View in Colaboratory](https://colab.research.google.com/github/iamrajee/Machine-Learning/blob/master/date_accuracy_check4.ipynb)

In [0]:
print("he")

In [21]:
!ls

datalab  drive


In [0]:
import pandas as pd
import numpy as np
y_train_df = pd.read_csv("drive/ml_data/transfer_learning/train_date_data/datefinal.csv")

In [0]:
y_all = y_train_df['0']
for i,ele in enumerate(y_all):
  y_all[i] = (ele-1900)//5

In [0]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
transfomed_label = encoder.fit_transform(y_all)
Y = transfomed_label

In [0]:
import keras

#
import os
import time
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from keras import optimizers
#


from keras import applications
from keras import optimizers
from keras import backend as k

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard,EarlyStopping

In [0]:
img_width, img_height = 256, 256

# batch_size = 16
# epochs = 50

model = applications.VGG19(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))


In [0]:
data_path = 'drive/ml_data/transfer_learning/image_data'

img_data_list=[]


from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

img_list=os.listdir(data_path)

img_list = sorted(img_list, key=lambda x: x)

for img in img_list:
	img_path = data_path + '/'+ img 
	img = image.load_img(img_path, target_size=(256, 256))
	x = image.img_to_array(img)
	x = np.expand_dims(x, axis=0)
	x = preprocess_input(x)
# 	print('Input image shape:', x.shape)
	img_data_list.append(x)
#   if i%100==0:
#     print(i)

In [9]:
img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

(910, 1, 256, 256, 3)
(1, 910, 256, 256, 3)
(910, 256, 256, 3)


In [0]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

num_classes = 16
num_of_samples = img_data.shape[0]

# print(np.shape(X_train))
# print(np.shape(X_test))
# print(np.shape(y_train))
# print(np.shape(y_test))

In [0]:
#freeze the few(1st five) layers for undesirable change

for layer in model.layers[:4]:
  layer.trainable = False


In [0]:
#adding custom layers
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024,activation="relu")(x)
predictions = Dense(num_classes,activation="softmax", name='output_layer')(x)

In [13]:
#create final model
model_final = Model(input=model.input, output = predictions)


# model_final.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ou...)`
  """Entry point for launching an IPython kernel.


In [14]:
model_final.compile(loss='categorical_crossentropy',optimizer= optimizers.RMSprop(lr=2e-4),metrics=['accuracy'])

t=time.time()
hist = model_final.fit(X_train, y_train, batch_size=8, epochs=30, verbose=1, validation_data=(X_test, y_test))
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = model_final.evaluate(X_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))


Train on 728 samples, validate on 182 samples
Epoch 1/30


ResourceExhaustedError: ignored

In [0]:
import matplotlib.pyplot as plt
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(12)

In [0]:
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('num of Epochs')
plt.ylabel('loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['classic'])